In [1]:
import numpy as np
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectPosition
from utils.orient import OrientUtils
from evaluate.evaluator import Evaluator
import loss_funcs

from utils.image import ImageUtils
from tqdm.auto import tqdm
from collections import defaultdict

In [2]:
INIT_LOCATION = (0, 1.3, 0.3)

LOSS_FUNCTIONS = [
    loss_funcs.IOU(),
    loss_funcs.MSE(),
    loss_funcs.RMSE(norm="euclidean"),
    loss_funcs.NMI(bins=50),
    loss_funcs.PSNR(),
    loss_funcs.SSIM(),
    loss_funcs.Hausdorff(),
    loss_funcs.ARE(),
    loss_funcs.VI(),
]


OBJECTS = ["airplane", "hammer", "hand", "headphones", "mouse", "mug", "stapler", "toothpaste"]

ZFAR = 5

In [3]:
def create_viewer(obj_name: str, is_sim: bool = True) -> ViewSampler:
    location = (INIT_LOCATION[0], INIT_LOCATION[2] - 1.3, INIT_LOCATION[2])
    cam_config = CameraConfig(location, rotation=(np.pi / 2, 0, 0), fov=30, zfar=ZFAR)
    if is_sim:
        viewer = ViewSampler(f"data/{obj_name}/world_sim.xml", cam_config)
    else:
        viewer = ViewSampler(f"data/{obj_name}/world.xml", cam_config)
    return viewer

In [4]:
def generate_positions(
    count: int,
) -> list[ObjectPosition]:
    orients = OrientUtils.generate_random(count)
    positions = [ObjectPosition(orient, INIT_LOCATION) for orient in orients]
    return positions

In [5]:
def get_views(
    viewer: ViewSampler, pos1: ObjectPosition, pos2: ObjectPosition, depth: bool
) -> tuple[np.ndarray, np.ndarray]:
    img1, _ = viewer.get_view_cropped(pos1, depth=depth)
    img2, _ = viewer.get_view_cropped(pos2, depth=depth)
    return img1, img2

In [6]:
def calc_penalty(num_samples):
    positions1 = generate_positions(num_samples)
    positions2 = generate_positions(num_samples)
    dists = {}
    for obj_name in tqdm(OBJECTS):
        total = 0
        count = 0
        dists[obj_name] = []
        with create_viewer(obj_name) as viewer:
            for pos1, pos2 in tqdm(zip(positions1, positions2), total=num_samples):
                img1, img2 = get_views(viewer, pos1, pos2, depth=True)
                pad_shape = np.maximum(img1.shape, img2.shape)
                img1 = ImageUtils.pad_to_shape(img1, pad_shape, pad_value=0)
                img2 = ImageUtils.pad_to_shape(img2, pad_shape, pad_value=0)
                both = (img1 > 0) & (img2 > 0)

                dists[obj_name].append(np.sum(np.abs(img1[both] - img2[both])) / np.sum(both))
                total += np.sum(np.abs(img1[both] - img2[both]))
                count += np.sum(both)
        print(f"Penalty for {obj_name}: {total / count}")
    return dists

In [7]:
dists = calc_penalty(1000)

  0%|          | 0/8 [00:00<?, ?it/s]

  wl_display@1 still attached


  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
for obj, dist_list in dists.items():
    print(f"=== {obj} ===")
    # print(type(dist_list[0]))
    # dist_list = np.asanyarray(dist_list)
    print(f"Mean: {np.mean(dist_list)}")
    print(f"Median: {np.median(dist_list)}")
    print(f"Max: {np.max(dist_list)}")
    print(f"Min: {np.min(dist_list)}")

In [ ]:
import matplotlib.pyplot as plt
import math
%matplotlib qt

def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw=None, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if ax is None:
        plt.figure()
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    for i in range(len(col_labels)):
        for j in range(len(row_labels)):
            text = ax.text(j, i, round(data[i, j],2),
                        ha="center", va="center", color="w")
            
    return im, cbar


In [ ]:
import numpy as np

n = 24

init_pos = np.array([0, 0, 0.3])
uni_orients = [list(init_pos)]

q = np.array([1, 0, 0])
delta = np.linspace(0, np.pi / 2, num=n + 1)

for i in range(n):
    uni_orients.append(list(init_pos + delta[i + 1] * q))

uni_orients = OrientUtils.generate_random(20)
uni_positions = [ObjectPosition(orient, INIT_LOCATION) for orient in uni_orients]
len(uni_positions)

In [ ]:
object_name = OBJECTS[1]
viewer = create_viewer(object_name, False)

In [ ]:
nrows, ncols = (4, 5)
fig, axes = plt.subplots(nrows, ncols)
for idx, pos in enumerate(uni_positions):
    col = idx % ncols
    row = math.floor(idx / ncols)
    ax = axes[row, col]
    ax.imshow(viewer.get_view_cropped(pos, depth=False)[0])
    # print(pos)
    ax.set_title(f"{idx}::{np.round(pos.orientation, 4)}")

In [ ]:
# Evaluate XorDiff
from evaluate import eval_funcs
from itertools import product

eval_func = eval_funcs.XorDiff(0.1)
eval_results = []

for pos1, pos2 in product(uni_positions, uni_positions):
    img1, img2 = get_views(viewer, pos1, pos2, depth=True)
    result = eval_func(img1, img2)
    eval_results.append(result)
results = np.array(eval_results).reshape(len(uni_positions), -1)

In [ ]:
import seaborn as sns

deltas = np.round(delta, 2)
fig, axes = plt.subplots(figsize=(13, 10))
sns.heatmap(results, ax=axes, annot=True, fmt=".2f", xticklabels=(deltas), yticklabels=deltas)
axes.set_title(f"XorDiff as function of rotation\n Initial orientation {init_pos}")
axes.set_xlabel("angel change (radians)")
axes.set_ylabel("angel change (radians)")

axes.tick_params(axis="x", rotation=90)
axes.tick_params(axis="y", rotation=0)
axes.autoscale()
fig.tight_layout()

In [ ]:
heatmap(results, range(len(uni_positions)), range(len(uni_positions)))

In [ ]:
import seaborn as sns

fig, ax = plt.subplots()
sns.lineplot(x=delta, y=results[0])

ax.set_title("XorDiff as function of rotation")
ax.set_xlabel("angel change (radians)")
ax.set_ylabel("XorDiff value")

In [7]:
N = 100
positions1 = generate_positions(N)
positions2 = generate_positions(N)

In [8]:
from evaluate import eval_funcs
import config

eval_func = eval_funcs.XorDiff(0.1)
eval_results = []

for obj_name in tqdm(OBJECTS):
    eval_func = eval_funcs.XorDiff(config.XORDIFF_PENALTY[obj_name])
    with create_viewer(obj_name) as sim_viewer:
        for pos1, pos2 in tqdm(zip(positions1, positions2), total=N):
            img1, img2 = get_views(sim_viewer, pos1, pos2, depth=True)
            result = eval_func(img1, img2)
            eval_results.append(result)

  0%|          | 0/8 [00:00<?, ?it/s]

  wl_display@1 still attached


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [9]:
loss_results = defaultdict(list)

for obj_name in tqdm(OBJECTS):
    with create_viewer(obj_name) as sim_viewer:
        for pos1, pos2 in tqdm(zip(positions1, positions2), total=N):
            img1, img2 = get_views(sim_viewer, pos1, pos2, depth=False)
            for loss_func in LOSS_FUNCTIONS:
                result = loss_func(img1, img2)
                loss_results[loss_func.get_name()].append(result)

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
# pearsons correlation
for k, loss_vals in loss_results.items():
    eval_vals = np.asanyarray(eval_results)
    loss_vals = np.asanyarray(loss_vals)
    print(k, np.corrcoef(eval_vals, loss_vals)[0, 1])

IOU 0.5850558814404224
MSE 0.2863102475842799
RMSE 0.505121388304349
NMI 0.506604833211717
PSNR 0.3085654264344736
SSIM -0.1244261672562479
Hausdorff 0.42737074209030085
ARE 0.15014054153411913
VI 0.41339695699363654


In [11]:
from scipy.stats import spearmanr, kendalltau

# spearmanr correlation
for k, loss_vals in loss_results.items():
    eval_vals = np.asanyarray(eval_results)
    loss_vals = np.asanyarray(loss_vals)
    print(f"{k}: spearman: {spearmanr(eval_vals, loss_vals).statistic} kendall: {kendalltau(eval_vals, loss_vals).statistic}")

IOU: spearman: 0.6312127204371377 kendall: 0.4412366101954871
MSE: spearman: 0.3285787946543667 kendall: 0.21538798498122655
RMSE: spearman: 0.5839511467986669 kendall: 0.41070713391739677
NMI: spearman: 0.5283557161808066 kendall: 0.3617146433041302
PSNR: spearman: 0.3285787946543667 kendall: 0.21538798498122655
SSIM: spearman: -0.13032749269920735 kendall: -0.0873404255319149
Hausdorff: spearman: 0.622763707443293 kendall: 0.431683354192741
ARE: spearman: 0.09896989682796381 kendall: 0.06794743429286608
VI: spearman: 0.4504409538139904 kendall: 0.3022528160200251


In [12]:
print(loss_results)

defaultdict(<class 'list'>, {'IOU': [0.7117977528089887, 0.5127829002514669, 0.6845738942826322, 0.6847487001733102, 0.6761498340445709, 0.6525901863566421, 0.6702516233766234, 0.7465419039869813, 0.6769825918762089, 0.7020345093999485, 0.8010710035700119, 0.7398547629218283, 0.5860976518822214, 0.6704980842911877, 0.6692061557214688, 0.4817749603803486, 0.7885779122541603, 0.6192482177576151, 0.6421267893660532, 0.7734164070612669, 0.6880040322580645, 0.6750691669228405, 0.7629609093429442, 0.634432466708941, 0.7529510961214165, 0.6571918647390346, 0.7444069227522161, 0.7263761879146495, 0.6338788594567235, 0.5660919540229885, 0.6648103028193526, 0.6759898292771522, 0.7924739195230999, 0.545995670995671, 0.6393474842767295, 0.5149527006408301, 0.7613422359185071, 0.8407265774378585, 0.5942418426103646, 0.7679979648944288, 0.7017779268701778, 0.5122216186854971, 0.8019525801952581, 0.7128712871287128, 0.7226950354609929, 0.7975206611570248, 0.7722852512155591, 0.717103026181571, 0.7346

In [13]:
from matplotlib import pyplot as plt

font = {"weight": "normal", "size": 10}

plt.rc("font", **font)
plt.rcParams["text.usetex"] = False

plt.cla()
fig, axes = plt.subplots(4, 3, sharex=False, figsize=(20, 20))


for i, (loss, values) in enumerate(loss_results.items()):
    ax = axes[i // 3, i % 3]
    ax.set_xlabel("Eval Error")
    ax.set_ylabel(loss)
    ax.set_title(f"{loss} Objective Function", fontweight="bold")

    x = eval_results
    y = np.polyval(np.polyfit(x, values, 1), x)

    ax.plot(x, values, ".", label=loss, markersize=6)
    ax.plot(x, y, ":", linewidth=5)
plt.show()

In [ ]:
from algs.algorithm import RunConfig
from algs.uniform_sampling import UniformSampling
from evaluate import eval_funcs

run_config = RunConfig(max_time=1000, silent=True)

eval_positions = generate_positions(100)

results = defaultdict(lambda: defaultdict(list))

SELECTED_LOSSES = [
    loss_funcs.IOU(),
    loss_funcs.RMSE(),
    loss_funcs.WeightedSum(loss_funcs.IOU(), loss_funcs.RMSE()),
    loss_funcs.NMI(50),
    loss_funcs.Hausdorff(),
]

LOSS_NAMES = [l.get_name() for l in SELECTED_LOSSES]

for obj_name in tqdm(OBJECTS):
    with create_viewer(obj_name, True) as sim_viewer, create_viewer(obj_name, False) as world_viewer:
        for loss_func, loss_name in zip(SELECTED_LOSSES, LOSS_NAMES):
            alg = UniformSampling(sim_viewer, loss_func=loss_func, num_samples=512)
            evaluator = Evaluator(world_viewer, sim_viewer, eval_func=eval_funcs.XorDiff(0.1))
            losses = evaluator.evaluate(alg, run_config, eval_positions)
            results[obj_name][loss_name].extend(losses)

In [ ]:
from utils.io import save_pickle

res_dict = dict()

for k, v in results.items():
    res_dict[k] = dict(v)

# save_pickle("plots/loss_eval_results.pkl", results)

In [ ]:
from matplotlib import pyplot as plt

font = {"weight": "bold", "size": 10}

plt.rc("font", **font)

fig, axes = plt.subplots(3, 3, sharex=False, figsize=(30, 30))

for i, object_name in enumerate(OBJECTS):
    ax = axes[i // 3, i % 3]
    ax.set_title(object_name.upper(), fontweight="bold", fontsize=15)

    ax.boxplot(results[object_name].values(), labels=LOSS_NAMES, sym="", patch_artist=False, autorange=True)
    # ax.set_ylabel("Eval Error")

In [ ]:
from matplotlib import pyplot as plt

font = {"weight": "bold", "size": 10}

plt.rc("font", **font)

labels = [l.get_name() for l in SELECTED_LOSSES]

fig, axes = plt.subplots(3, 3, sharex=False, figsize=(30, 30))

medianprops = dict(linestyle=None, linewidth=0)
whiskerprops = dict(linewidth=0)
capprops = dict(linewidth=0)

for i, object_name in enumerate(OBJECTS):
    ax = axes[i // 3, i % 3]
    ax.set_title(object_name.upper(), fontweight="bold", fontsize=15)
    bp = ax.boxplot(
        results[object_name].values(),
        widths=0.1,
        sym="",
        patch_artist=True,
        # whis=[5, 95],
        # bootstrap=10000,
        medianprops=medianprops,
        whiskerprops=whiskerprops,
        capprops=capprops,
    )

    violin = ax.violinplot(results[object_name].values(), showmeans=False)
    ax.set_xticks([y + 1 for y in range(len(labels))], labels=LOSS_NAMES)